In [19]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
from scipy.stats import shapiro
from scipy.stats.mstats import winsorize
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [20]:
df = pd.read_csv('./dataset/label지정.csv', encoding='CP949')
df

,회사명,거래소코드,회계년도_x,상장일,상장폐지일,산업명,소속코드,[제조]매출액증가율,[제조]유형자산증가율,[제조]비유동자산증가율,...,[제조]기계투자효율,[제조]부가가치율,[제조]노동소득분배율,[제조]자본분배율,[제조]이윤분배율,[제조]유형자산회전율,[제조]이자보상배율(이자비용),year,month,label
0,(주)CMG제약,58820,2011-12-01,2001-08-31,NaN,의료용 물질 및 의약품 제조업,5,56.40,-5.90,-12.89,...,128.51,19.56,89.82,10.18,-43.44,1.77,3.53,2011,12,0
1,(주)CMG제약,58820,2012-12-01,2001-08-31,NaN,의료용 물질 및 의약품 제조업,5,-8.43,-4.12,-16.43,...,10.67,1.66,1749.87,-1649.87,-2473.07,1.71,-7.75,2012,12,0
2,(주)CMG제약,58820,2013-12-01,2001-08-31,NaN,의료용 물질 및 의약품 제조업,5,25.51,16.09,77.05,...,289.40,34.29,71.28,28.72,4.23,2.03,0.70,2013,12,0
3,(주)CMG제약,58820,2014-12-01,2001-08-31,NaN,의료용 물질 및 의약품 제조업,5,11.52,74.30,68.15,...,132.06,31.04,79.30,20.70,2.24,1.54,3.60,2014,12,0
4,(주)CMG제약,58820,2015-12-01,2001-08-31,NaN,의료용 물질 및 의약품 제조업,5,18.85,-5.37,1.66,...,90.92,15.96,144.10,-44.10,-84.58,1.48,-32.22,2015,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10480,흥구석유(주),24060,2016-12-01,1994-12-07,NaN,도매 및 상품 중개업,5,-7.26,-5.61,0.11,...,3684.02,4.86,48.08,51.92,31.14,3.67,7.66,2016,12,0
10481,흥구석유(주),24060,2017-12-01,1994-12-07,NaN,도매 및 상품 중개업,5,8.12,-0.56,-0.48,...,4786.26,4.39,47.63,52.37,30.24,4.10,5.91,2017,12,0
10482,흥구석유(주),24060,2018-12-01,1994-12-07,NaN,도매 및 상품 중개업,5,6.87,-0.36,-2.09,...,3064.10,4.44,47.47,52.53,32.14,4.40,12.66,2018,12,0
10483,흥구석유(주),24060,2019-12-01,1994-12-07,NaN,도매 및 상품 중개업,5,-9.35,-7.00,-2.63,...,3489.21,10.00,22.19,77.81,55.02,4.14,252.02,2019,12,0


In [21]:
df_1 = df.copy()
df_1.drop(['회사명', '거래소코드', '회계년도_x', '상장일', '상장폐지일', '산업명', '소속코드','year', 'month'],axis=1,inplace=True)
for col in df_1.columns:
    df_1[col] = winsorize(df_1[col], limits= 0.01)

In [51]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split


feature = df_1.drop(columns='label')
target = df_1['label']

#릿지 클래서
ridge = Ridge(alpha = 10)
ridge_alphas =[0,0.1,1,10,100]

for alpha in ridge_alphas :
    ridge = Ridge(alpha = alpha)

# fig, axs = plt.subplots(figsize = (18,6), nrows = 1, ncols=5)

coeff_df = pd.DataFrame()

for pos, alpha in enumerate(ridge_alphas):
    ridge = Ridge(alpha= alpha)
    ridge.fit(feature,target)
    coeff = pd.Series(data=ridge.coef_, index=feature.columns)
    colname = 'alpha:' + str(alpha)
    coeff_df[colname] = coeff

    coeff = coeff.sort_values(ascending=False)
#     axs[pos].set_title(colname)
#     axs[pos].set_xlim(-3,6)
#     sns.barplot(x=coeff.values, y=coeff.index, ax =axs[pos])

# plt.show()

In [ ]:
ridge_alphas =[0,0.1,1,10,100]
sort_column = 'alpha:'+str(ridge_alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)

In [80]:
lasso = Lasso(alpha = 10)
alphas =[0.25]


# fig, axs = plt.subplots(figsize = (18,6), nrows = 1, ncols=5)

coeff_df = pd.DataFrame()

for pos, alpha in enumerate(alphas):
    lasso = Lasso(alpha = alpha)
    lasso.fit(feature,target)
    coeff = pd.Series(data=lasso.coef_, index=feature.columns)
    colname = 'alpha:' + str(alpha)
    coeff_df[colname] = coeff

    coeff = coeff.sort_values(ascending=False)

In [ ]:
lasso_alphas =[0.25]
sort_column = 'alpha:'+str(lasso_alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)


In [115]:

a = coeff_df[coeff_df["alpha:0.25"]<-0.0000000001] 
b = coeff_df[coeff_df["alpha:0.25"]>0.00000001]

a.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, [제조]비유동자산증가율 to [제조]기계투자효율
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   alpha:0.2   10 non-null     float64
 1   alpha:0.25  10 non-null     float64
 2   alpha:0.3   10 non-null     float64
 3   alpha:0.35  10 non-null     float64
 4   alpha:0.4   10 non-null     float64
dtypes: float64(5)
memory usage: 480.0+ bytes


In [123]:
feature = pd.concat([a,b])
feature = feature["alpha:0.25"]
feature = pd.DataFrame(feature)
feature

,alpha:0.25
[제조]비유동자산증가율,-1.101468e-05
[제조]재고자산증가율,-1.642654e-05
[제조]매출액순이익률,-2.140773e-04
[제조]자기자본순이익률,-1.477270e-04
[제조]1주당매출액,-6.825699e-08
[제조]유동비율,-5.829955e-06
[제조]자기자본배율,-2.152229e-06
[제조]부가가치,-1.208220e-07
[제조]설비투자효율,-1.393240e-06
[제조]기계투자효율,-4.364865e-09


In [118]:
# ridge_alphas =[0,0.1,0.2,0.3,0.4,0.5]
# sort_column = 'alpha:'+str(ridge_alphas)
# coeff_df.sort_values(by=sort_column, ascending=False)

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np              
# from sklearn.metrics import mean_squared_error

# plt.scatter(Y_train,Y_pred_train, label = '(실제값, 예측값)', alpha=.5)
# plt.plot(np.linspace(0,20,200), np.linspace(0,50,200), color = 'green', label = "45°(실제값 = 예측값)")
# plt.xlabel("실제 Price: $Y_i$")
# plt.ylabel("예측 Price : $\hat{Y}_i$")
# plt.legend()
# plt.title("예측된 Price of train set vs 실제 Price ($Y_i$ vs $\hat{Y}_i$), " + f"alpha={alpha_value}")
# plt.text(y=0,x=35, s=f'MSE 값 : {mean_squared_error(Y_train,Y_pred_train) : .4f}')
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# from sklearn.metrics import mean_squared_error

# plt.scatter(Y_test,Y_pred, label = '(실제값, 예측값)', alpha=1)
# plt.plot(np.linspace(0,50,200), np.linspace(0,50,200), color = 'green', label = "45°(실제값 = 예측값)")
# plt.xlabel("실제 Price: $Y_i$")
# plt.ylabel("예측 Price: $\hat{Y}_i$")
# plt.legend()
# plt.title("예측 Price of test set vs 실제 Price ($Y_i$ vs $\hat{Y}_i$), " + f"alpha={alpha_value}")
# plt.text(y=0,x=35, s=f'MSE 값 : {mean_squared_error(Y_test,Y_pred) : .4f}')
# plt.show()

In [ ]:
# # 잔차 분석
# resid = Y_pred_train - Y_train

# plt.scatter(Y_pred_train, resid, c='b')
# plt.hlines(y=0, xmin= -10, xmax=50)
# plt.title('Residual plot')
# plt.show()

In [12]:
# 정규성 검정 (귀무가설 : 정규성를 따른다)
from scipy.stats import jarque_bera

statistic, pvalue = jarque_bera(resid)
print(f'검정통계치 : {statistic : .5f}\np값 : {pvalue : .5f}')

검정통계치 :  1405424.09835
p값 :  0.00000


In [13]:
# 등분산 검정 (귀무가설 : 오차항이 등분산성을 따른다)
from statsmodels.stats.api import het_breuschpagan

# Bresuch-Pagan test
names = ['Lag_range multiplier statistic', 'p-value','f-value', 'f p-value']
test = het_breuschpagan(resid, X_train)

# 독립변수, featue의 values (endog는 y값)

for i in zip(names, test):
    print(f'{i[0]} : {i[1] : .3f}')

Lag_range multiplier statistic :  866.598
p-value :  0.000
f-value :  20.019
f p-value :  0.000


In [ ]:
from sklearn.linear_model import Lasso

def get_linear_reg_eval(model_name, paramas = None, X_data_n =None, y_target_n=None, verbose=True):
    coeff_df = pd.DataFrame()
    if verbose